### open deepcode response

In [1]:
# Основные библиотеки
import pandas as pd
import torch
from transformers import BertModel, BertTokenizer
from tqdm import tqdm  # Импорт tqdm для отображения прогресса
import numpy as np
import json
import os
import re
import io
import ast
import time
import tempfile
import subprocess
from collections import Counter
from multiprocessing import Process, Queue
from contextlib import redirect_stdout
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import matplotlib.pyplot as plt
import requests
from cleantext import clean
from better_profanity import profanity


path_to_test_data = "../data/for_teams/test"

In [2]:
gigachat_response = pd.read_csv('../data/deepseek/submit_deepseek.csv')

In [3]:
df_solutions_test = pd.read_excel(f'{path_to_test_data}/solutions.xlsx')
df_tests_test = pd.read_excel(f'{path_to_test_data}/tests.xlsx')
df_tasks_test = pd.read_excel(f'{path_to_test_data}/tasks.xlsx')

In [4]:
if 'problem_message' not in df_solutions_test.columns:
    df_solutions_test['problem_message'] = ''
if 'line_with_code' not in df_solutions_test.columns:
    df_solutions_test['line_with_code'] = ''
if 'error_type' not in df_solutions_test.columns:
    df_solutions_test['error_type'] = ''

In [5]:
for index, row in df_solutions_test.iterrows():
    try:
        ast.parse(row['student_solution'])
    except SyntaxError as e:
        df_solutions_test.at[index, 'problem_message'] = str(e)
        df_solutions_test.at[index, 'error_type'] = type(e).__name__
        
        # Получаем строку кода, вызвавшую ошибку
        lines = row['student_solution'].split('\n')
        if e.lineno is not None and 1 <= e.lineno <= len(lines):
            df_solutions_test.at[index, 'line_with_code'] = lines[e.lineno - 1]
        else:
            df_solutions_test.at[index, 'line_with_code'] = ''

In [6]:
def prepare_data(df_solutions, df_tests, df_tasks):
    # 1 Проверяем, что код компилируется
    if "code_problem" not in df_solutions.columns:
        df_solutions["code_problem"] = False

    # Проходим по всему DataFrame и проверяем код в student_solution
    for index, row in df_solutions.iterrows():
        try:
            ast.parse(row["student_solution"])
        except SyntaxError as e:
            df_solutions.at[index, "code_problem"] = True

    # 2 Удаляем комментарии
    def remove_cyrillic_comments(text):
        # Регулярное выражение для поиска комментариев с кириллицей
        pattern = r"#[^#\n]*[\u0400-\u04FF]"
        # Замена найденных комментариев на пустую строку
        return re.sub(pattern, "", text)

    # Применение функции к столбцу student_solution
    df_solutions["student_solution"] = df_solutions["student_solution"].apply(remove_cyrillic_comments)

    # 3 Выполняем тесты
    def replace_input_with_values(code, input_values):
        # Разделяем код на строки
        code_lines = code.split('\n')

        # Если input_values — строка, разделяем её на отдельные значения
        if isinstance(input_values, str):
            input_values_list = input_values.split('\n')
        else:
            input_values_list = input_values

        input_value_index = 0
        inside_loop = False
        new_code_lines = []

        is_loop_input = False
        for line in code_lines:
            # Проверяем, начался ли цикл
            if re.search(r'\bwhile\b|\bfor\b', line):
                inside_loop = True

            # Если находим вызов input()
            if 'input()' in line and inside_loop:
                is_loop_input = True

            # Проверяем конец цикла
            if inside_loop and re.search(r'\bbreak\b|\bcontinue\b|\bpass\b|\breturn\b', line):
                inside_loop = False

        inside_loop = False
        # if is_loop_input:
        #     new_code_lines.extend('')

        input_before_loop_cnt = 0
        for line in code_lines:
            # print(line, '|', inside_loop)
            # Проверяем, начался ли цикл
            if re.search(r'\bwhile\b|\bfor\b', line):
                inside_loop = True
                if is_loop_input:
                    new_code_lines.append(f'input_index = -1')
                    new_code_lines.append(f'input_values = {input_values_list[input_before_loop_cnt:]}')

            # Если находим вызов input()
            if 'input()' in line:
                if is_loop_input and inside_loop:
                    new_code_lines.append(f'    input_index += 1')
                    value = 'input_values[input_index]'
                    is_loop_input = False
                else:
                    if is_loop_input and not inside_loop:
                        input_before_loop_cnt += 1
                    if input_value_index < len(input_values_list):
                        value = f'"{input_values_list[input_value_index]}"'
                        input_value_index += 1
                    else:
                        value = '""'  # Если не хватает входных данных

                # Заменяем input() на текущее значение
                line = re.sub(r'input\s*\(\s*\)', f'{value}', line, count=1)

            new_code_lines.append(line)

            # Проверяем конец цикла
            if inside_loop and re.search(r'\bbreak\b|\bcontinue\b|\bpass\b|\breturn\b', line):
                inside_loop = False

        return '\n'.join(new_code_lines)

    def execute_code_with_timeout(code, input_values, timeout=0.5):
        def target(queue, code, input_values):
            # Заменяем вызовы input() на передачу данных напрямую
            code = replace_input_with_values(code, input_values)
            # Перенаправляем стандартный вывод в буфер
            buffer = io.StringIO()
            with redirect_stdout(buffer):
                try:
                    exec(code)
                except Exception as e:
                    queue.put(f"Error: {str(e)}")
                    return

            # Получаем результат вывода
            output = buffer.getvalue().strip()
            queue.put(output)

        q = Queue()
        p = Process(target=target, args=(q, code, input_values))
        p.start()
        p.join(timeout)

        if p.is_alive():
            p.terminate()
            p.join()
            return "Timeout"
        else:
            return q.get()

    def checkout_tests(solutions, tests):
        # Добавляем колонки error_open и error_closed со значением False по умолчанию        
        solutions['error_open_tests'] = False
        solutions['error_closed_tests'] = False
        solutions['test_info'] = [[] for _ in range(len(solutions))]  # Новая колонка для хранения информации о тестах

        # Проходим по каждой строке в solutions с использованием tqdm для отображения прогресса
        for index, row in tqdm(solutions.iterrows(), total=solutions.shape[0], desc="Processing solutions"):
            if not row['code_problem']:
                task_id = row['task_id']
                student_solution = row['student_solution']

                # Фильтруем tests по task_id
                relevant_tests = tests[tests['task_id'] == task_id]

                # Проходим по каждому тесту с использованием tqdm для отображения прогресса
                for test_index, test_row in relevant_tests.iterrows():

                    test_id = test_row['id']
                    test_input = test_row['input']
                    expected_output = test_row['output']
                    test_type = test_row['type']

                    try:
                        actual_output = execute_code_with_timeout(student_solution, test_input)
                        # print(actual_output)
                        # Сравниваем результат с ожидаемым выводом
                        if "Timeout" not in actual_output and "Error" not in actual_output:
                            if actual_output != expected_output:
                                if test_type == 'open':
                                    solutions.at[index, 'error_open_tests'] = True
                                elif test_type == 'closed':
                                    solutions.at[index, 'error_closed_tests'] = True

                                # Добавляем информацию о тесте в колонку test_info
                                test_info = [test_input, expected_output, actual_output]
                                solutions.at[index, 'test_info'].append(test_info)
                    except Exception as e:
                        pass

        return solutions

    for task_id in df_tasks['id']:
        df_solutions.loc[df_solutions['task_id'] == task_id, 'description'] = df_tasks.loc[df_tasks['id'] == task_id, 'description'].values[0]
        df_solutions.loc[df_solutions['task_id'] == task_id, 'author_solution'] = df_tasks.loc[df_tasks['id'] == task_id, 'author_solution'].values[0]
    df_solutions = checkout_tests(df_solutions, df_tests)
    return df_solutions

In [7]:
df_solutions_test = prepare_data(df_solutions_test, df_tests_test, df_tasks_test)
df_solutions_test.head(10)

Processing solutions: 100%|██████████| 325/325 [00:56<00:00,  5.73it/s]


,id,task_id,student_solution,author_comment,author_comment_embedding,problem_message,line_with_code,error_type,code_problem,description,author_solution,error_open_tests,error_closed_tests,test_info
0,0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,print(f'Реализация проекта будет стоить {money...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,print(fРеализация проекта будет стоить {money}...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
3,3,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
4,4,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
5,5,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,"print(f""Реализация проекта будет стоить {money...",SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
6,6,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
7,7,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"unexpected indent (<unknown>, line 6)",print(f'Реализация проекта будет стоить {m...,IndentationError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
8,8,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"'(' was never closed (<unknown>, line 6)",print(f'Реализация проекта будет стоить {money...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]
9,9,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[]


In [8]:
def create_test_requests(df_solutions_test, df_gigachat):
    def remove_empty_lines(text):
        return "\n".join([line for line in text.splitlines() if line.strip()])

    def create_request(row, df_gigachat_row):
        # Удаляем пустые строки в описании задачи
        description = remove_empty_lines(row['description'])
        req = f"Есть задача, описанная так: ${description}$."

        # Удаляем пустые строки в решении студента
        student_solution = remove_empty_lines(row['student_solution'])
        if student_solution:
            req += f" У меня есть решение: ${student_solution}$."

        if row["code_problem"]:
            # Удаляем пустые строки в сообщении об ошибке
            problem_message = remove_empty_lines(row['problem_message'])
            if problem_message:
                req += f" В коде есть ошибка: ${problem_message}$."
        else:
            if row["test_info"]:
                req += " Оно не проходит тесты."
                if len(row["test_info"]) > 0:
                    test = row["test_info"][0]                
                    if test[0] and test[1] and test[2]:
                        req += f" При вводе {test[0]} ожидалось {test[1]}, но я получаю {test[2]}.\n"

        req += "Используй этот ответ для формирования ответа: " + df_gigachat_row

        return req.replace('\n', '')

    # Создаем список словарей в нужном формате
    dataset_for_GPT = []

    system_message = {
        "role": "system",
        "text": (
        "Вы — профессиональный программист и наставник, задача которого — помогать другим улучшать свой код."
        "Ваша задача — давать краткие советы по исправлению ошибок, максимум в 3 предложения, не давая готового решения."
        "Дайте краткую подсказку для исправления ошибки, объясняя её в контексте задания. Предоставляйте только рекомендации, фокусируясь на логике."
        "Используйте в речи такие фразы, как: Ваш код, выполняет условия, попробуйте изменить, условия задания, некорректно выполняет, скорректировать ошибку, забыли поставить, задания например, условие if."
        "А также: ваш код некорректно, выполняет условия задания, код некорректно выполняет, ошибку ваш код, изменить условие if. "
        "Общайтесь уважительно, на Вы."
        ),
    }

    # Создаем список для датафрейма
    df_list = []

    # Добавляем новую колонку 'requests' в df_solutions_test
    if 'system_message' not in df_solutions_test.columns:    
        df_solutions_test['system_message'] = None
    if 'request_text' not in df_solutions_test.columns:    
        df_solutions_test['request_text'] = None

    for index, row in df_solutions_test.iterrows():
        request_text = create_request(row, df_gigachat.iloc[index]['author_comment'])
        dataset_for_GPT.append(
            {"request": [system_message, {"role": "user", "text": request_text}], "response": row["author_comment"]}
        )

        # Добавляем данные в список для датафрейма
        df_list.append({"system_message": system_message["text"], "request_text": request_text})

        # Заполняем новую колонку 'requests'
        df_solutions_test.at[index, 'system_message'] = system_message["text"]
        df_solutions_test.at[index, 'request_text'] = request_text

    # Создаем датафрейм из списка
    df_system_request_test = pd.DataFrame(df_list)

    # Сохраняем датафрейм в CSV файл
    df_system_request_test.to_excel("system_request_pairs.xlsx")

    # Сохраняем список словарей в JSON файл
    with open("dataset_for_GPT_testDEEPYA.json", "w", encoding="utf-8") as f:
        for entry in dataset_for_GPT:
            json.dump(entry, f, ensure_ascii=False)
            f.write("\n")

    # Возвращаем обновленный датафрейм
    return df_solutions_test

In [9]:
df_solutions_test = create_test_requests(df_solutions_test, gigachat_response)
print(df_solutions_test.iloc[0]['request_text'])

Есть задача, описанная так: $Реализуйте программу, которая напечатает стоимость реализации проекта со скидкой: В переменную dicount считывается размер скидки на реализацию проекта (вещественное число). Этот код уже написан.В переменную money считывается стоимость реализации проекта в тысячах (целое число). Этот код уже написан.Программа должна вывести фразу вида 'Реализация проекта будет стоить <стоимость> тыс. руб. без скидки. Со скидой стоимость составит <стоимость с учетом скидки> тыс. руб.'. Важно! В коде обязательно нужно использовать f-строку!$. У меня есть решение: $discount  = float(input())money = int(input())print('Реализация проекта будет стоить {money} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.')$.Используй этот ответ для формирования ответа:  Вы некорректно выполняете условия задания, используя переменную `money` в двух местах в f-строке. Вместо этого нужно использовать переменную `discount` для вычисления стоимости с учетом ск

In [10]:
# Функция для подсчета символов в строке
def count_characters(text):
    return len(text)

# Функция для подсчета символов в списке строк
def count_characters_in_list(text_list):
    return sum(count_characters(text) for text in text_list)

# Открываем JSON файл и читаем его содержимое
with open('dataset_for_GPT_testDEEPYA.json', 'r', encoding='utf-8') as f:
    lines = f.readlines()

max_request_characters = 0
max_response_characters = 0
request_exceeds_3999_count = 0

# Проходим по каждой строке JSON файла
for line in lines:
    entry = json.loads(line)
    
    # Подсчитываем символы в request
    request_characters = count_characters_in_list([msg['text'] for msg in entry['request']])
    if request_characters > max_request_characters:
        max_request_characters = request_characters
    
    # Проверяем, превышает ли request 3999 символов
    if request_characters > 3999:
        request_exceeds_3999_count += 1
    
    # Подсчитываем символы в response
    response_characters = count_characters(entry['response'])
    if response_characters > max_response_characters:
        max_response_characters = response_characters

print(f"Максимальное количество символов в request: {max_request_characters}")
print(f"Максимальное количество символов в response: {max_response_characters}")
print(f"Количество раз, когда request превышает 3999 символов: {request_exceeds_3999_count}")

Максимальное количество символов в request: 3810
Максимальное количество символов в response: 17
Количество раз, когда request превышает 3999 символов: 0


In [11]:
df_solutions_test.head(10)

,id,task_id,student_solution,author_comment,author_comment_embedding,problem_message,line_with_code,error_type,code_problem,description,author_solution,error_open_tests,error_closed_tests,test_info,system_message,request_text
0,0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,print(f'Реализация проекта будет стоить {money...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,print(fРеализация проекта будет стоить {money}...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
3,3,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
4,4,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
5,5,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,"print(f""Реализация проекта будет стоить {money...",SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
6,6,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
7,7,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"unexpected indent (<unknown>, line 6)",print(f'Реализация проекта будет стоить {m...,IndentationError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
8,8,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"'(' was never closed (<unknown>, line 6)",print(f'Реализация проекта будет стоить {money...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."
9,9,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ..."


In [12]:
# Загрузка моделей
print("Loading models...", end="")
model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
print("OK")

# Функция для получения эмбединга предложения
def get_sentence_embedding(sentence: str) -> torch.Tensor:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding

# Функция для преобразования строки в эмбединг
def string2embedding(string: str) -> torch.Tensor:
    return torch.Tensor([float(i) for i in string.split()])

# Функция для преобразования эмбединга в строку
def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])

# Функция для вычисления косинусного расстояния между двумя векторами

def cosine_distance(a, b):
    # Проверка и преобразование строк в numpy.array
    if isinstance(a, str):
        a = np.array(a.split(), dtype=float)       
    if isinstance(b, str):
        b = np.array(b.split(), dtype=float)
    
    # Проверка, что a и b являются numpy.array
    if not isinstance(a, np.ndarray) or not isinstance(b, np.ndarray):
        raise ValueError("Аргументы должны быть numpy.array или строками, которые можно преобразовать в numpy.array")
    
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return 1 - (dot_product / (norm_a * norm_b))

# Функция для вычисления среднего косинусного расстояния
def mean_cosine_distance(embeddings):
    n = len(embeddings)
    distances = []
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = cosine_distance(embeddings[i], embeddings[j])
            distances.append(dist)
    
    return np.mean(distances)

Loading models...OK


In [13]:
file_name = '../jailbreak/obscene_lexicon.txt'
with open(file_name, 'r', encoding='utf-8') as file:
    bad_words = file.read().split('\n')

def clean_answer(text):
    def remove_code_from_text(text):
        code_pattern = re.compile(r'\b(def|class|if|else|elif|for|while|try|except|finally|with|import|from|print|return|break|continue|pass|raise|assert|yield|async|await|lambda|global|nonlocal|del|exec|eval)\b.*', re.DOTALL)
        fenced_code_pattern = re.compile(r'```.*?```', re.DOTALL)
        cleaned_text = code_pattern.sub('', text)
        cleaned_text = fenced_code_pattern.sub('', cleaned_text)
        return cleaned_text

    def remove_obscene_lexicon(text):
        cleaned_text = clean(text, extra_spaces=True, stemming=False, stopwords=True, lowercase=False, stp_lang='english' )
        cleaned_text = profanity.censor(cleaned_text)
        for word in bad_words:
            bad_words_re = re.compile(r'\b(' + word + r')\b', re.IGNORECASE)
            cleaned_text = bad_words_re.sub(lambda match: '***', cleaned_text)
        return cleaned_text

    text = remove_code_from_text(text)
    text = remove_obscene_lexicon(text)
    return text

In [14]:
if 'response' not in df_solutions_test.columns:
    df_solutions_test['response'] = None
if 'resp_lenght' not in df_solutions_test.columns:
    df_solutions_test['resp_lenght'] = None
if 'sentence_cnt' not in df_solutions_test.columns:
    df_solutions_test['sentence_cnt'] = None
if 'author_comment_embedding' not in df_solutions_test.columns:
    df_solutions_test['author_comment_embedding'] = False 

In [15]:
def get_predictions(predictions_needed_cnt):
    # URL для запроса к API Yandex Cloud
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

    # Заголовки запроса
    headers = {
        "Content-Type": "application/json",
        "Authorization": "t1.9euelZqcxpickpfHzJjLns6ZypWLxu3rnpWanMqLjIybi4mSkZuYi5yaypfl8_dWQi1H-e85RF4n_t3z9xZxKkf57zlEXif-zef1656VmpHMx4qQlpSKms2bx8uNxouX7_zF656VmpHMx4qQlpSKms2bx8uNxouX.T_V3SdGXXuQxJ7zt8ychz8tMXIv71KH5ytRbVesR9m280NDlqJ8bHxp7as6xTGkuaVq06a1WDq6_R3IuYwguBw",
        "x-folder-id": "b1g9kv27t3avpjnp1gpf"
    }

    # Счетчик реально отправленных запросов
    real_request_count = 0

    # Счетчик неудачных запросов
    failed_request_count = 0

    def send_request_and_save_response(row):
        nonlocal real_request_count, failed_request_count

        # Проверка, есть ли уже данные в столбце 'response'
        if pd.isna(row['response']):
            # Извлечение данных из датафрейма
            system_message = row['system_message']
            request_text = row['request_text']

            # Формирование данных для запроса
            data = {
                "modelUri": "ds://bt1mk8ar7svnotmmhh78",
                "completionOptions": {
                    "stream": False,
                    "temperature": 0.1,
                    "maxTokens": "2000"
                },
                "messages": [
                    {
                        "role": "system",
                        "text": system_message
                    },
                    {
                        "role": "user",
                        "text": request_text
                    }
                ]
            }

            while True:
                try:
                    # Отправка POST-запроса
                    response = requests.post(url, headers=headers, data=json.dumps(data))
                    response.raise_for_status()  # Проверка на ошибки HTTP

                    response_json = response.json()
                    if 'result' in response_json and 'alternatives' in response_json['result']:
                        message_text = clean_answer(response_json['result']['alternatives'][0]['message']['text'])
                        real_request_count += 1
                    else:
                        message_text = clean_answer(response_json)

                    # Сохранение ответа в столбец 'author_comment'
                    df_solutions_test.at[index, 'author_comment'] = message_text

                    # Вычисление эмбединга текста
                    text_embedding = embedding2string(get_sentence_embedding(message_text))
                    df_solutions_test.at[index, 'author_comment_embedding'] = text_embedding

                    # Вычисление количества символов в тексте
                    resp_lenght = len(message_text)
                    df_solutions_test.at[index, 'resp_lenght'] = resp_lenght

                    # Вычисление количества предложений в тексте
                    sentence_cnt = len(message_text.split('.'))
                    df_solutions_test.at[index, 'sentence_cnt'] = sentence_cnt

                    # Возвращаем текст сообщения
                    return message_text
                except requests.exceptions.RequestException as e:
                    if response.status_code == 429:  # "Too Many Requests"
                        print(f"Получен ответ 'Too Many Requests'. Повторная попытка через 1 секунду...")
                        time.sleep(1)  # Пауза перед повторной попыткой
                    else:
                        print(f"Ошибка при отправке запроса: {e}")
                        failed_request_count += 1
                        if failed_request_count >= 3:
                            print("Превышено количество неудачных запросов. Остановка процесса.")
                            return None
                        return None
        else:
            # Если данные уже есть, возвращаем их
            return row['response']

    # Отправка запросов только для первых N строк, где нет данных в столбце 'response'
    for index, row in tqdm(df_solutions_test.iterrows(), total=df_solutions_test.shape[0], desc="Processing"):
        if real_request_count >= predictions_needed_cnt:
            break
        df_solutions_test.at[index, 'response'] = send_request_and_save_response(row)
        time.sleep(0.1)  # Таймаут между запросами

    print(f"Количество реально отправленных запросов: {real_request_count}")
    print(f"Количество неудачных запросов: {failed_request_count}")

In [16]:
predictions_needed_cnt = 1
get_predictions(predictions_needed_cnt)
df_solutions_test['response'].isna().sum()

Processing:   0%|          | 0/325 [00:00<?, ?it/s]

Ошибка при отправке запроса: 401 Client Error: Unauthorized for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion


Processing:   0%|          | 1/325 [00:00<00:51,  6.31it/s]

Ошибка при отправке запроса: 401 Client Error: Unauthorized for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion


Processing:   1%|          | 2/325 [00:00<00:50,  6.35it/s]

...

Processing:  99%|█████████▉| 323/325 [00:51<00:00,  6.43it/s]

Ошибка при отправке запроса: 401 Client Error: Unauthorized for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Превышено количество неудачных запросов. Остановка процесса.
Ошибка при отправке запроса: 401 Client Error: Unauthorized for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Превышено количество неудачных запросов. Остановка процесса.


Processing: 100%|██████████| 325/325 [00:51<00:00,  6.33it/s]

Ошибка при отправке запроса: 401 Client Error: Unauthorized for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Превышено количество неудачных запросов. Остановка процесса.
Количество реально отправленных запросов: 0
Количество неудачных запросов: 325


325

In [17]:
if df_solutions_test['response'].isna().sum() == 0:
    df_solutions_test.to_csv('after_model.csv')
assert df_solutions_test['response'].isna().sum() == 0

AssertionError: 

In [ ]:
df_solutions_test.head(10)

,id,task_id,student_solution,author_comment,author_comment_embedding,problem_message,line_with_code,error_type,code_problem,description,author_solution,error_open_tests,error_closed_tests,test_info,system_message,request_text,response,resp_lenght,sentence_cnt
0,0,0,discount = float(input())\nmoney = int(input(...,Ваш код некорректно выполняет условия задания....,-0.4242990016937256 -1.488830804824829 0.10249...,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",Ваш код некорректно выполняет условия задания....,140,3
1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,print(f'Реализация проекта будет стоить {money...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",None,None,None
2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,print(fРеализация проекта будет стоить {money}...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",None,None,None
3,3,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",None,None,None
4,4,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",None,None,None
5,5,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,unterminated string literal (detected at line ...,"print(f""Реализация проекта будет стоить {money...",SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",None,None,None
6,6,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",None,None,None
7,7,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"unexpected indent (<unknown>, line 6)",print(f'Реализация проекта будет стоить {m...,IndentationError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",None,None,None
8,8,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"'(' was never closed (<unknown>, line 6)",print(f'Реализация проекта будет стоить {money...,SyntaxError,True,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы — профессиональный программист и наставник,...","Есть задача, описанная так: $Реализуйте програ...",None,None,None
9,9,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,,,False,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...,False,False,[],"Вы

In [ ]:
# Функция для логирования
def log_action(message):
    print(f"LOG: {message}")

# Проходим по строкам обоих датафреймов
for i in range(len(df_solutions_test)):
    error_open_tests = df_solutions_test.loc[i, 'error_open_tests']
    error_closed_tests = df_solutions_test.loc[i, 'error_closed_tests']
    
    if error_open_tests and error_closed_tests:
        df_solutions_test.at[i, 'author_comment'] = "Ошибка в скрытых и открытых тестах.\n\n " + df_solutions_test.at[i, 'response']
        log_action(f"Row {i}: Added 'Ошибка в скрытых и открытых тестах.' to response")
    elif error_open_tests:
        df_solutions_test.at[i, 'author_comment'] = "Ошибка в открытых тестах.\n\n " + df_solutions_test.at[i, 'response']
        log_action(f"Row {i}: Added 'Ошибка в открытых тестах.' to response")
    elif error_closed_tests:
        df_solutions_test.at[i, 'author_comment'] = "Ошибка в скрытых тестах.\n\n " + df_solutions_test.at[i, 'response']
        log_action(f"Row {i}: Added 'Ошибка в скрытых тестах.' to response")


In [66]:
df_solutions_test = pd.read_csv('after_model.csv')

In [67]:
for index, row in tqdm(df_solutions_test.iterrows(), total=df_solutions_test.shape[0]):
    author_comment = row['author_comment'] 
    author_comment_embedding = embedding2string(get_sentence_embedding(author_comment)) 
    df_solutions_test.at[index, 'author_comment_embedding'] = author_comment_embedding

  0%|          | 0/325 [00:00<?, ?it/s]

100%|██████████| 325/325 [00:35<00:00,  9.11it/s]


In [68]:
# Выбираем нужные колонки
selected_df = df_solutions_test[['id', 'author_comment', 'author_comment_embedding']]

# Переименовываем колонку 'id' в 'solution_id'
selected_df.rename(columns={'id': 'solution_id'}, inplace=True)

# Сохраняем результат в CSV файл
selected_df.to_csv('../data/solutions/submit_finetunedYA-m0.4-DEEP-v3.csv', index=False)

/tmp/ipykernel_38081/488833951.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df.rename(columns={'id': 'solution_id'}, inplace=True)


In [69]:
TEST_SIZE = 325
EMBEDDING_SIZE = 768


def _check_ids_correctness(submit_df: pd.DataFrame, submit_example_df: pd.DataFrame) -> bool:
    not_presented = set(submit_example_df["solution_id"]) - set(submit_df["solution_id"])
    not_needed = set(submit_df["solution_id"]) - set(submit_example_df["solution_id"])

    not_presented = list(not_presented)
    not_presented.sort()
    not_needed = list(not_needed)
    not_needed.sort()

    error_message = "Submit is incorrect."
    if len(not_presented) + len(not_needed) > 0:
        if len(not_presented) > 0:
            error_message += f" Not presented solution_id: {not_presented}."
        if len(not_needed) > 0:
            error_message += f" Not needed solution_id: {not_needed}."
        raise ValueError(error_message)
    return True


def _check_rows_size_correctness(submit_df: pd.DataFrame) -> bool:
    incorrect_rows = []
    for idx in range(TEST_SIZE):
        if len(string2embedding(submit_df["author_comment_embedding"].iloc[idx])) != EMBEDDING_SIZE:
            incorrect_rows.append(idx)
    if len(incorrect_rows) > 0:
        raise ValueError(f"Submit has incorrect rows: {incorrect_rows}. (incorrect size of embedding)")
    return True


def check_submit_correctness(submit_path: str, submit_example_path: str) -> bool:
    if not submit_path.endswith(".csv"):
        raise ValueError(f"{submit_path} is not a .csv file.")

    submit_df = pd.read_csv(submit_path)
    submit_example_df = pd.read_csv(submit_example_path)

    _check_ids_correctness(submit_df, submit_example_df)
    _check_rows_size_correctness(submit_df)

    return True

check_submit_correctness(submit_path="../data/solutions/submit_finetunedYA-m0.4-DEEP-v3.csv", submit_example_path="../data/for_teams/submit_example.csv")  

True

In [70]:
model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

#information that we have parsed form linter info
confirmed_errors = pd.read_excel('../data/new_data/data_with_tests_info_test18_10.xlsx')
current_solution = pd.read_csv("../data/solutions/submit_finetunedYA-m0.4-DEEP-v3.csv")
current_solution['result_linter_ru'] = confirmed_errors['result_linter_ru']
current_solution['author_comment'] = current_solution.apply(lambda x: x['result_linter_ru'] if not pd.isna(x['result_linter_ru']) else x['author_comment'], axis = 1)

def get_sentence_embedding(sentence: str) -> torch.Tensor:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding

def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])

current_solution['author_comment_embedding'] = current_solution['author_comment'].apply(lambda x: embedding2string(get_sentence_embedding(x)))

current_solution[["solution_id", "author_comment", "author_comment_embedding"]].to_excel('submit_finetunedYA-m0.4-DEEP-v3_2.xlsx')
current_solution[["solution_id", "author_comment", "author_comment_embedding"]].to_csv("../data/solutions/submit_finetunedYA-m0.4-DEEP-v3_2.csv", index=False)

print("Final solution is ready")